<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/draft/football_live_experimental_heft_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project config

In [1]:
try:
    import neptune.new as neptune
except:
    !pip install neptune-client >> None
    import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

In [2]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Installations

In [3]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20443 sha256=02e1b39d7803ad66aea12c51864de2ac9b57970a9369e8393de02e276319dc90
  Stored in directory: /root/.cache/pip/wheels/6a/65/c4/d27eeee9ba3fc150a0dae150519591103b9e0dbffde3ae77dc
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3530 sha256=8d2a5e77913d0968eb81a93403ff7702a2f2a3362bf1c3a33600c280d92c8ce8
  Stored in directory: /root/.cache/pip/wheels/ba/d9/a0/ddd93af16d5855dd9bad417623e70948fdac119d1d34fb17c8
Successfully built fst-pso miniful
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0

### Downloads

In [4]:
data_version = 'football_live_221229/'
project = neptune.init_project(
    name="scomesse/football", 
    api_token = api_key
    )
project[data_version + 'data.rar'].download('./data.rar')
project[data_version + 'info.rar'].download('./info.rar')
project[data_version + 'prem.rar'].download('./prem.rar')
project[data_version + 'train_id.csv'].download('./train_id.csv')
project[data_version + 'test_id.csv'].download('./test_id.csv')
project[data_version + 'holdout_id.csv'].download('./holdout_id.csv')
project[data_version + 'data_train.csv.gz'].download('./data_train.csv.gz')
project[data_version + 'data_test.csv.gz'].download('./data_test.csv.gz')
project[data_version + 'data_hold.csv.gz'].download('./data_hold.csv.gz')

#del project[data_version]
project.stop()

https://app.neptune.ai/scomesse/football/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api/project#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/metadata


### Imports

In [3]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

#import dask.dataframe as dd
import subprocess
import sys
from glob import glob
from tqdm import tqdm

1.3.5
1.21.6


In [4]:
import plotly.express as px

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

### Code

#####Functions

In [6]:
def run_bash(bashCommand:str, nameCommand = ''):
        process = subprocess.Popen([bashCommand], 
                           shell=True)
        _, error = process.communicate()
        if error:
            print(f'{nameCommand} error:\n', error)

#### Unpack

In [9]:
for file in glob('./*'):    
    if '.rar' in file:
        print(file)
        bash = f'unrar e {file} && rm {file}'
        run_bash(bash)

./prem.rar
./info.rar
./data.rar


### Preprocessing & Feature engineering

In [7]:
data_df = pd.read_csv(
    './data_test.csv.gz', 
    dtype = {
        'Id':np.int32, 'Minute':np.int16, 'Active':np.int8,
        'Score1':np.int16, 'Score2':np.int16, 'A1':np.int16, 'A2':np.int16,
        'DA1':np.int16, 'DA2':np.int16,'Pos1':np.float32, 'Pos2':np.float32,
        'Off1':np.int16, 'Off2':np.int16,'On1':np.int16, 'On2':np.int16,
        'YC1':np.float32, 'YC2':np.float32, 'RC1':np.float32, 'RC2':np.float32,
        'Sub1':np.float32, 'Sub2':np.float32, 'Pen1':np.float32, 'Pen2':np.float32,
        'Cor1':np.float32, 'Cor2':np.float32,
        }
    )

In [8]:
data_df = data_df[data_df['Id'] != 1898989]

In [9]:
# Присоединяем итоговый результат и по первому тайму
def add_match_results (data_df:pd.DataFrame, cols = ['Id', 'Result1', 'Result2', 'Periods'], info_path = './info.csv'):
    info_df = pd.read_csv(info_path, sep = ';', usecols = cols)
    info_df[['Period1', 'Period2', 'Period3', 'Period4']] = info_df['Periods'].str.split(',', expand = True)
    info_df[['Time1Res1', 'Time1Res2']] = info_df['Period1'].str.split(':', expand = True)
    info_df = info_df[~info_df['Id'].duplicated(keep = False)]
    period1_result_dict = info_df.set_index('Id')[['Time1Res1', 'Time1Res2', 'Result1', 'Result2']].to_dict(orient = 'index')
    #data_df.loc[:, ['Time1Res1', 'Time1Res2', 'Result1', 'Result2']] =  \
    return [[period1_result_dict[id]['Time1Res1'], period1_result_dict[id]['Time1Res1'], 
    period1_result_dict[id]['Result1'], period1_result_dict[id]['Result2']] 
    if id in period1_result_dict else [None, None, None, None] for id in tqdm(data_df['Id'].values, total = len(data_df))]
    

In [10]:
data_df.loc[:, ['Time1Res1', 'Time1Res2', 'Result1', 'Result2']] = add_match_results (data_df)
isdigit = [str(a) for a in range(20)]
data_df.loc[~data_df['Time1Res1'].isin(isdigit), ['Time1Res1']] = -1
data_df.loc[~data_df['Time1Res2'].isin(isdigit), ['Time1Res2']] = -1
data_df[['Time1Res1', 'Time1Res2']] = data_df[['Time1Res1', 'Time1Res2']].astype(np.int16)
data_df[['Result1', 'Result2']] = data_df[['Result1', 'Result2']].astype(np.int16)
data_df = data_df.drop(['Time1Res1', 'Time1Res2'], axis = 1)

100%|██████████| 2598283/2598283 [00:32<00:00, 78757.46it/s] 


In [11]:
# Добавляем прематчевые линии
def add_match_lines (data_df:pd.DataFrame, cols = ['P1', 'PX', 'P2', 'PR'], prem_path = './prem.csv'):
    prem_df = pd.read_csv(prem_path, sep = ';')
    prem_df = prem_df[~prem_df['Id'].duplicated(keep = False)]
    prem_dict = prem_df.set_index('Id')[cols].to_dict(orient = 'index')
    #data_df.loc[:,['P1', 'PX', 'P2', 'PR']] =  \
    return [[prem_dict[id]['P1'], prem_dict[id]['PX'], prem_dict[id]['P2'], prem_dict[id]['PR']] 
    if id in prem_dict else [None, None, None, None] for id in tqdm(data_df['Id'].values, total = len(data_df))]

In [12]:
data_df.loc[:,['P1', 'PX', 'P2', 'PR']] = add_match_lines (data_df)
data_df[['P1', 'PX', 'P2', 'PR']] = data_df[['P1', 'PX', 'P2', 'PR']].astype(np.float32)
data_df = data_df.drop(['PX', 'PR'], axis = 1)

100%|██████████| 2598283/2598283 [00:28<00:00, 89991.70it/s]


In [13]:
def transform_dataset(data_df:pd.DataFrame, remain = []):

    # трансформируем минуты
    data_df['min_norm'] = data_df['Minute'].astype(np.float32) / 50
    print('1. Минуты посчитаны...')
    # трансформируем голы
    data_df['Score1_norm'] = data_df['Score1'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score1'] > 3, ['Score1_norm']] = 1.0
    data_df['Score2_norm'] = data_df['Score2'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score2'] > 3, ['Score2_norm']] = 1.0

    data_df['Score_diff'] = data_df['Score1'].astype(np.int16) - data_df['Score2'].astype(np.int16)
    data_df.loc[data_df['Score_diff'] < -4, ['Score_diff']] = -4
    data_df.loc[data_df['Score_diff'] > 4, ['Score_diff']] = 4
    data_df[[f'Score_cat_{n}' for n in range(1, 10)]] = pd.get_dummies(data_df['Score_diff']).values
    data_df['Score_diff'] = data_df['Score_diff'].astype(np.float32) / np.float32(4.0)
    if 'Score1' not in remain:
        data_df = data_df.drop(['Score1', 'Score2'], axis = 1)
    print('2. Голы посчитаны...')
    #трансформируем атаки
    data_df['A1_scaled'] = data_df['A1'].astype(np.float32) / 75
    data_df.loc[data_df['A1'] >= 60, ['A1_scaled']] = (60 + (data_df['A1'] - 60) / 4) / 75
    data_df['A2_scaled'] = data_df['A2'].astype(np.float32) / 75
    data_df.loc[data_df['A2'] >= 60, ['A2_scaled']] = (60 + (data_df['A2'] - 60) / 4) / 75
    # атаки в минуту
    data_df['A1perMIN'] = data_df['A1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A1perMIN'] > 4, ['A1perMIN']] = np.float32(4.0)
    data_df['A2perMIN'] = data_df['A2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A2perMIN'] > 4, ['A2perMIN']] = np.float32(4.0)
    # динамика атак
    data_df['A1relativ'] = data_df['A1'].astype(np.float32) - data_df['A1'].shift(5).astype(np.float32)
    data_df.loc[data_df['A1relativ'] < 0, ['A1relativ']] = np.float32(0.0)
    data_df['A1relativ'] = data_df['A1relativ'].fillna(0)
    data_df.loc[data_df['A1relativ'] > 15, ['A1relativ']] = np.float32(15.)
    data_df['A2relativ'] = data_df['A2'].astype(np.float32) - data_df['A2'].shift(5).astype(np.float32)
    data_df.loc[data_df['A2relativ'] < 0, ['A2relativ']] =  np.float32(0.0)
    data_df['A2relativ'] = data_df['A2relativ'].fillna(0)
    data_df.loc[data_df['A2relativ'] > 15, ['A2relativ']] = np.float32(15.)
    if 'A1' not in remain:
        data_df = data_df.drop(['A1', 'A2'], axis = 1)
    print('3. Атаки посчитаны...')
    # трансформируем опасные атаки
    data_df['DA1_scaled'] = data_df['DA1'].astype(np.float32) / 50
    data_df.loc[data_df['DA1'] >= 40, ['DA1_scaled']] = (80 + (data_df['DA1'] - 40) / 3) / 100
    data_df['DA2_scaled'] = data_df['DA2'].astype(np.float32) / 50
    data_df.loc[data_df['DA2'] >= 40, ['DA2_scaled']] = (80 + (data_df['DA2'] - 40) / 3) / 100
    # опасные атаки в минуту    
    data_df['DA1perMIN'] = data_df['DA1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA1perMIN'] > 3, ['DA1perMIN']] = np.float32(3.0)
    data_df['DA2perMIN'] = data_df['DA2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA2perMIN'] > 3, ['DA2perMIN']] = np.float32(3.0)
    # динамика опасных атак
    data_df['DA1relativ'] = data_df['DA1'].astype(np.float32) - data_df['DA1'].shift(5).astype(np.float32)
    data_df.loc[data_df['DA1relativ'] < 0, ['DA1relativ']] = np.float32(0.0)
    data_df['DA1relativ'] = data_df['DA1relativ'].fillna(0)
    data_df.loc[data_df['DA1relativ'] > 10, ['DA1relativ']] = np.float32(10.)
    data_df['DA2relativ'] = data_df['DA2'].astype(np.float32) - data_df['DA2'].shift(5).astype(np.float32)
    data_df.loc[data_df['DA2relativ'] < 0, ['DA2relativ']] = np.float32(0.0)
    data_df['DA2relativ'] = data_df['DA2relativ'].fillna(0)
    data_df.loc[data_df['DA2relativ'] > 10, ['DA2relativ']] = np.float32(10.)
    if 'DA1' not in remain:
        data_df = data_df.drop(['DA1', 'DA2'], axis = 1)
    if 'Minute' not in remain:
        data_df = data_df.drop(['Minute'], axis = 1)
    print('4. Опасные атаки посчитаны...')
    # трансформируем владение мячом
    data_df['Pos1_cleaned'] = data_df['Pos1'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos1_cleaned'] < 0.2, ['Pos1_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos1_cleaned'] > 0.8, ['Pos1_cleaned']] = np.float32(0.8)
    data_df['Pos2_cleaned'] = data_df['Pos2'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos2_cleaned'] < 0.2, ['Pos2_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos2_cleaned'] > 0.8, ['Pos2_cleaned']] = np.float32(0.8)
    if 'Pos1' not in remain:
        data_df = data_df.drop(['Pos1', 'Pos2'], axis = 1)
    print('5. Владение мячом посчитпно...')
    # трансформируем удары
    data_df['Off1_norm'] = data_df['Off1'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off1_norm'] > 1.0, ['Off1_norm']] = np.float32(1.0)
    data_df['Off2_norm'] = data_df['Off2'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off2_norm'] > 1.0, ['Off2_norm']] = np.float32(1.0)
    if 'Off1' not in remain:
        data_df = data_df.drop(['Off1', 'Off2'], axis = 1)
    print('6. Удары посчитаны...')
    # трансформируем удары в створ
    data_df['On1_norm'] = data_df['On1'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On1_norm'] > 1.0, ['On1_norm']] = np.float32(1.0)
    data_df['On2_norm'] = data_df['On2'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On2_norm'] > 1.0, ['On2_norm']] = np.float32(1.0)
    if 'On1' not in remain:
        data_df = data_df.drop(['On1', 'On2'], axis = 1)    
    print('7. Удары в створ посчитаны...')
    # трансформируем желтые карточки
    data_df['YC1_transformed'] = data_df['YC1'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC1_transformed'] > 1.0, ['YC1_transformed']] = np.float32(1.0)
    data_df['YC2_transformed'] = data_df['YC2'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC2_transformed'] > 1.0, ['YC2_transformed']] = np.float32(1.0)
    if 'YC1' not in remain:
        data_df = data_df.drop(['YC1', 'YC2'], axis = 1)
    print('8. Жёлтые карточки посчитаны...')
    # трансформируем красные карточки
    data_df['RC1_transformed'] = data_df['RC1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC1_transformed'] > 1, ['RC1_transformed']] = np.int8(1)
    data_df['RC2_transformed'] = data_df['RC2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC2_transformed'] > 1, ['RC2_transformed']] = np.int8(1)
    if 'RC1' not in remain:
        data_df = data_df.drop(['RC1', 'RC2'], axis = 1)
    print('9. Красные карточки посчитаны...')
    # трансформируем замены
    data_df['Sub1_transformed'] = data_df['Sub1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub1_transformed'] > 1, ['Sub1_transformed']] = np.int8(1)
    data_df['Sub2_transformed'] = data_df['Sub2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub2_transformed'] > 1, ['Sub2_transformed']] = np.int8(1)
    if 'Sub1' not in remain:
        data_df = data_df.drop(['Sub1', 'Sub2'], axis = 1)
    print('10. Замены посчитаны...')
    # трансформируем угловык
    data_df['Cor1_transformed'] = data_df['Cor1'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor1_transformed'] > 1.0, ['Cor1_transformed']] = np.float32(1.0)
    data_df['Cor2_transformed'] = data_df['Cor2'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor2_transformed'] > 1.0, ['Cor2_transformed']] = np.float32(1.0)
    if 'Cor1' not in remain:
        data_df = data_df.drop(['Cor1', 'Cor2'], axis = 1)
    print('11. Угловые посчитаны...')
    # трансформируем линию
    data_df['P1_transformed'] = np.log(data_df['P1'], dtype = np.float32) / 2
    data_df['P2_transformed'] = np.log(data_df['P2'], dtype = np.float32) / 2
    if 'P1' not in remain:
        data_df = data_df.drop(['P1', 'P2'], axis = 1)
    print('12. Линии посчитаны...')
    if 'Pen1' not in remain:
        data_df = data_df.drop(['Pen1', 'Pen2'], axis = 1)
    if 'Active' not in remain:
        data_df = data_df.drop(['Active'], axis = 1)
    return data_df

In [14]:
data_df = transform_dataset(
    data_df, 
    #remain = 
    #    ['Id', 'Minute', 'Active', 'Score1', 'Score2', 'A1', 'A2', 'DA1', 'DA2',
    #   'Pos1', 'Pos2', 'Off1', 'Off2', 'On1', 'On2', 'YC1', 'YC2', 'RC1',
    #   'RC2', 'Sub1', 'Sub2', 'Pen1', 'Pen2', 'Cor1', 'Cor2', 'P1', 'P2']
       ) #, remain = ['Id', 'Time1Res1', 'Time1Res2', 'Result1', 'Result2'])

1. Минуты посчитаны...
2. Голы посчитаны...
3. Атаки посчитаны...
4. Опасные атаки посчитаны...
5. Владение мячом посчитпно...
6. Удары посчитаны...
7. Удары в створ посчитаны...
8. Жёлтые карточки посчитаны...
9. Красные карточки посчитаны...
10. Замены посчитаны...
11. Угловые посчитаны...
12. Линии посчитаны...


In [15]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2598283 entries, 0 to 2598282
Data columns (total 47 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Id                int32  
 1   Active            int8   
 2   Pen1              float32
 3   Pen2              float32
 4   Result1           int16  
 5   Result2           int16  
 6   min_norm          float32
 7   Score1_norm       float32
 8   Score2_norm       float32
 9   Score_diff        float32
 10  Score_cat_1       uint8  
 11  Score_cat_2       uint8  
 12  Score_cat_3       uint8  
 13  Score_cat_4       uint8  
 14  Score_cat_5       uint8  
 15  Score_cat_6       uint8  
 16  Score_cat_7       uint8  
 17  Score_cat_8       uint8  
 18  Score_cat_9       uint8  
 19  A1_scaled         float32
 20  A2_scaled         float32
 21  A1perMIN          float32
 22  A2perMIN          float32
 23  A1relativ         float32
 24  A2relativ         float32
 25  DA1_scaled        float32
 26  DA2_scaled    

In [ ]:
data_df.describe()

,A2relativ,Score_cat_7,Score_diff,On1_norm,Off1_norm,Cor1_transformed,Cor2_transformed,Pos1_cleaned,Score_cat_2,min_norm,DA1relativ,Sub1_transformed,Off2_norm,YC1_transformed,On2_norm,Score1_norm,Score_cat_6,DA2relativ,A2_scaled,DA2perMIN,Score_cat_8,A1relativ,YC2_transformed,A1_scaled,Pos2_cleaned,Score_cat_9,P2_transformed,A2perMIN,Score2_norm,DA2_scaled,DA1_scaled,RC2_transformed,Sub2_transformed,Score_cat_3,DA1perMIN,RC1_transformed,Score_cat_4,Score_cat_1,P1_transformed,A1perMIN,Score_cat_5
count,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.030882e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.039200e+07,1.030882e+07,1.039200e+07,1.039200e+07
mean,4.772809e+00,3.234883e-02,1.616325e-02,2.015467e-01,1.427330e-01,1.935282e-01,1.664271e-01,5.135605e-01,3.818515e-03,4.580627e-01,2.862621e+00,2.894121e-02,1.240342e-01,1.129467e-01,8.643056e-02,7.841690e-02,1.572950e-01,2.593980e+00,2.405378e-01,5.442840e-01,6.449674e-03,4.987762e+00,1.219509e-01,2.515548e-01,4.860962e-01,1.808314e-03,5.906410e-01,1.083414e+00,6.225887e-02,1.256547e-01,1.391360e-01,6.288300e-03,2.955976e-02,2.136769e-02,6.031695e-01,5.251829e-03,1.260131e-01,9.294652e-04,8.617290e-01,1.132017e+00,6.499694e-01
std,3.023146e+00,1.769248e-01,1.984648e-01,2.451463e-01,1.619019e-01,2.283660e-01,2.102087e-01,1.184697e-01,6.167604e-02,2.597798e-01,2.298063e+00,1.695528e-01,1.480107e-01,2.449198e-01,1.165000e-01,1.528303e-01,3.640787e-01,2.190674e+00,1.555736e-01,3.201990e-01,8.005046e-02,3.115531e+00,2.541701e-01,1.618362e-01,1.184298e-01,4.248582e-02,3.048857e-01,4.388057e-01,1.349634e-01,9.831203e-02,1.059510e-01,8.302585e-02,1.712616e-01,1.446068e-01,3.340472e-01,7.660798e-02,3.318642e-01,3.047296e-02,5.425605e-01,4.479147e-01,4.769792e-01
min,0.000000e+00,0.000000e+00,-1.000000e+00,-2.000000e-01,-1.000000e-01,-1.666667e-01,-1.666667e-01,2.000000e-01,0.000000e+00,-2.000000e-02,0.000000e+00,-1.000000e+00,-1.000000e-01,-5.000000e-01,-1.000000e-01,-2.500000e-01,0.000000e+00,0.000000e+00,-1.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,-5.000000e-01,-1.000000e-02,2.000000e-01,0.000000e+00,4.997736e-04,0.000000e+00,-2.500000e-01,-1.000000e-02,-1.000000e-02,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,9.995472e-04,0.000000e+00,0.000000e+00
25%,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.400000e-01,0.000000e+00,2.400000e-01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.100000e-01,3.333333e-01,0.000000e+00,3.000000e+00,0.000000e+00,1.200000e-01,4.100000e-01,0.000000e+00,3.942287e-01,8.095238e-01,0.000000e+00,5.000000e-02,6.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,3.809524e-01,0.000000e+00,0.000000e+00,0.000000e+00,5.007753e-01,8.518519e-01,0.000000e+00
50%,5.000000e+00,0.000000e+00,0.000000e+00,2.000000e-01,1.000000e-01,1.666667e-01,1.666667e-01,5.100000e-01,0.000000e+00,4.600000e-01,3.000000e+00,0.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,2.200000e-01,5.000000e-01,0.000000e+00,5.000000e+00,0.000000e+00,2.300000e-01,4.900000e-01,0.000000e+00,5.493062e-01,1.000000e+00,0.000000e+00,1.100000e-01,1.200000e-01,0.000000e+00,0.000000e+00,0.000000e+00,5.625000e-01,0.000000e+00,0.000000e+00,0.000000e+00,7.654679e-01,1.071429e+00,1.000000e+00
75%,7.000000e+00,0.000000e+00,0.000000e+00,4.000000e-01,2.000000e-01,3.333333e-01,3.333333e-01,5.900000e-01,0.000000e+00,6.800000e-01,4.000000e+00,0.000000e+00,2.000000e-01,0.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,4.000000e+00,3.400000e-01,7.1

In [ ]:
data_df.to_csv('data_train_transformed.csv.gz', index = False, compression={'method': 'gzip'})

### Catboost

#### Installation

In [16]:
!pip install catboost >> None

#### Imports

In [17]:
from catboost import CatBoost
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
np.random.seed(147)

#### Create Tensors

In [29]:
y_train = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[:2_000_000]
y_test = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[2_000_000:]

In [32]:
X_train = data_df.drop(['Id', 'Active', 'Pen1', 'Pen2', 'Result1', 'Result2'], axis = 1).values[:2_000_000,: ]
X_test = data_df.drop(['Id', 'Active', 'Pen1', 'Pen2', 'Result1', 'Result2'], axis = 1).values[2_000_000:,: ]

In [35]:
train_data = Pool(X_train, y_train)
test_data = Pool(X_test, y_test)

In [36]:
booster = CatBoostClassifier(iterations=10)

In [37]:
booster.fit(train_data, eval_set=test_data)

Learning rate set to 0.5
0:	learn: 0.5821645	test: 0.5842479	best: 0.5842479 (0)	total: 757ms	remaining: 6.81s
1:	learn: 0.5567266	test: 0.5598999	best: 0.5598999 (1)	total: 1.42s	remaining: 5.69s
2:	learn: 0.5491680	test: 0.5529460	best: 0.5529460 (2)	total: 2.02s	remaining: 4.7s
3:	learn: 0.5462579	test: 0.5504348	best: 0.5504348 (3)	total: 2.6s	remaining: 3.9s
4:	learn: 0.5445273	test: 0.5485606	best: 0.5485606 (4)	total: 3.15s	remaining: 3.15s
5:	learn: 0.5433533	test: 0.5476657	best: 0.5476657 (5)	total: 3.63s	remaining: 2.42s
6:	learn: 0.5425328	test: 0.5471024	best: 0.5471024 (6)	total: 4.19s	remaining: 1.79s
7:	learn: 0.5420255	test: 0.5467666	best: 0.5467666 (7)	total: 4.75s	remaining: 1.19s
8:	learn: 0.5415818	test: 0.5466984	best: 0.5466984 (8)	total: 5.35s	remaining: 594ms
9:	learn: 0.5411477	test: 0.5468835	best: 0.5466984 (8)	total: 5.85s	remaining: 0us

bestTest = 0.5466984427
bestIteration = 8

Shrink model to first 9 iterations.


In [38]:
print("\nTest  Accuracy : %.2f"%booster.score(train_data))
print("Train Accuracy : %.2f"%booster.score(test_data))


Test  Accuracy : 0.72
Train Accuracy : 0.71


In [ ]:
drop_transformed = True

In [ ]:
data_df['min_norm'] = data_df['Minute'].astype(np.float32) / 50

In [ ]:
data_df['Score1_norm'] = data_df['Score1'].fillna(method = 'ffill').astype(np.float32) / 4
data_df.loc[data_df['Score1'] > 3, ['Score1_norm']] = 1.0
data_df['Score2_norm'] = data_df['Score2'].fillna(method = 'ffill').astype(np.float32) / 4
data_df.loc[data_df['Score2'] > 3, ['Score2_norm']] = 1.0

data_df['Score_diff'] = data_df['Score1'].astype(np.float32) - data_df['Score2'].astype(np.float32)
data_df.loc[data_df['Score_diff'] < -4, ['Score_diff']] = -4
data_df.loc[data_df['Score_diff'] > 4, ['Score_diff']] = 4
data_df[[f'Score_cat_{n}' for n in range(1, 10)]] = pd.get_dummies(data_df['Score_diff']).values
data_df['Score_diff'] = data_df['Score_diff'].astype(np.float32) / np.float32(4.0)

if drop_transformed:
    data_df = data_df.drop(['Score1', 'Score2'], axis = 1)

In [ ]:
data_df['A1_scaled'] = data_df['A1'].astype(np.float32) / 100
data_df.loc[data_df['A1'] >= 80, ['A1_scaled']] = (80 + (data_df['A1'] - 80) / 5) / 100
data_df['A2_scaled'] = data_df['A2'].astype(np.float32) / 100
data_df.loc[data_df['A2'] >= 80, ['A2_scaled']] = (80 + (data_df['A2'] - 80) / 5) / 100

data_df['A1perMIN'] = data_df['A1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
data_df.loc[data_df['A1perMIN'] > 4, ['A1perMIN']] = np.float32(4.0)
data_df['A2perMIN'] = data_df['A2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
data_df.loc[data_df['A2perMIN'] > 4, ['A2perMIN']] = np.float32(4.0)

data_df['A1relativ'] = data_df['A1'].astype(np.float32) - data_df['A1'].shift(5).astype(np.float32)
data_df.loc[data_df['A1relativ'] < 0, ['A1relativ']] = np.float32(0.0)
data_df['A1relativ'] = data_df['A1relativ'].fillna(0)
data_df.loc[data_df['A1relativ'] > 15, ['A1relativ']] = np.float32(15.)
data_df['A2relativ'] = data_df['A2'].astype(np.float32) - data_df['A2'].shift(5).astype(np.float32)
data_df.loc[data_df['A2relativ'] < 0, ['A2relativ']] =  np.float32(0.0)
data_df['A2relativ'] = data_df['A2relativ'].fillna(0)
data_df.loc[data_df['A2relativ'] > 15, ['A2relativ']] = np.float32(15.)

if drop_transformed:
    data_df = data_df.drop(['A1', 'A2'], axis = 1)

In [ ]:
data_df['DA1_scaled'] = data_df['DA1'].astype(np.float32) / 100
data_df.loc[data_df['DA1'] >= 80, ['DA1_scaled']] = (80 + (data_df['DA1'] - 80) / 5) / 100
data_df['DA2_scaled'] = data_df['DA2'].astype(np.float32) / 100
data_df.loc[data_df['DA2'] >= 80, ['DA2_scaled']] = (80 + (data_df['DA2'] - 80) / 5) / 100

data_df['DA1perMIN'] = data_df['DA1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
data_df.loc[data_df['DA1perMIN'] > 3, ['DA1perMIN']] = np.float32(3.0)
data_df['DA2perMIN'] = data_df['DA2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
data_df.loc[data_df['DA2perMIN'] > 3, ['DA2perMIN']] = np.float32(3.0)

data_df['DA1relativ'] = data_df['DA1'].astype(np.float32) - data_df['DA1'].shift(5).astype(np.float32)
data_df.loc[data_df['DA1relativ'] < 0, ['DA1relativ']] = np.float32(0.0)
data_df['DA1relativ'] = data_df['DA1relativ'].fillna(0)
data_df.loc[data_df['DA1relativ'] > 10, ['DA1relativ']] = np.float32(10.)
data_df['DA2relativ'] = data_df['DA2'].astype(np.float32) - data_df['DA2'].shift(5).astype(np.float32)
data_df.loc[data_df['DA2relativ'] < 0, ['DA2relativ']] = np.float32(0.0)
data_df['DA2relativ'] = data_df['DA2relativ'].fillna(0)
data_df.loc[data_df['DA2relativ'] > 10, ['DA2relativ']] = np.float32(10.)

if drop_transformed:
    data_df = data_df.drop(['DA1', 'DA2'], axis = 1)

In [ ]:
data_df['Pos1_cleaned'] = data_df['Pos1'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
data_df.loc[data_df['Pos1_cleaned'] < 0.2, ['Pos1_cleaned']] = np.float32(0.2)
data_df.loc[data_df['Pos1_cleaned'] > 0.8, ['Pos1_cleaned']] = np.float32(0.8)
data_df['Pos2_cleaned'] = data_df['Pos2'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
data_df.loc[data_df['Pos2_cleaned'] < 0.2, ['Pos2_cleaned']] = np.float32(0.2)
data_df.loc[data_df['Pos2_cleaned'] > 0.8, ['Pos2_cleaned']] = np.float32(0.8)

if drop_transformed:
    data_df = data_df.drop(['Pos1', 'Pos2'], axis = 1)

In [ ]:
data_df['Off1_norm'] = data_df['Off1'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
data_df.loc[data_df['Off1_norm'] > 1.0, ['Off1_norm']] = np.float32(1.0)
data_df['Off2_norm'] = data_df['Off2'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
data_df.loc[data_df['Off2_norm'] > 1.0, ['Off2_norm']] = np.float32(1.0)

if drop_transformed:
    data_df = data_df.drop(['Off1', 'Off2'], axis = 1)

In [39]:
data_df['On1_norm'] = data_df['On1'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
data_df.loc[data_df['On1_norm'] > 1.0, ['On1_norm']] = np.float32(1.0)
data_df['On2_norm'] = data_df['On2'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
data_df.loc[data_df['On2_norm'] > 1.0, ['On2_norm']] = np.float32(1.0)

if drop_transformed:
    data_df = data_df.drop(['On1', 'On2'], axis = 1)

In [ ]:
data_df['YC1_transformed'] = data_df['YC1'].fillna(0).astype(np.float32) / np.float32(2.0)
data_df.loc[data_df['YC1_transformed'] > 1.0, ['YC1_transformed']] = np.float32(1.0)
data_df['YC2_transformed'] = data_df['YC2'].fillna(0).astype(np.float32) / np.float32(2.0)
data_df.loc[data_df['YC2_transformed'] > 1.0, ['YC2_transformed']] = np.float32(1.0)

if drop_transformed:
    data_df = data_df.drop(['YC1', 'YC2'], axis = 1)

In [ ]:
data_df['RC1_transformed'] = data_df['RC1'].fillna(0).astype(np.int8)
data_df.loc[data_df['RC1_transformed'] > 1, ['RC1_transformed']] = np.int8(1)
data_df['RC2_transformed'] = data_df['RC2'].fillna(0).astype(np.int8)
data_df.loc[data_df['RC2_transformed'] > 1, ['RC2_transformed']] = np.int8(1)

if drop_transformed:
    data_df = data_df.drop(['RC1', 'RC2'], axis = 1)

In [ ]:
data_df['Sub1_transformed'] = data_df['Sub1'].fillna(0).astype(np.int8)
data_df.loc[data_df['Sub1_transformed'] > 1, ['Sub1_transformed']] = np.int8(1)
data_df['Sub2_transformed'] = data_df['Sub2'].fillna(0).astype(np.int8)
data_df.loc[data_df['Sub2_transformed'] > 1, ['Sub2_transformed']] = np.int8(1)

if drop_transformed:
    data_df = data_df.drop(['Sub1', 'Sub2'], axis = 1)

In [ ]:
data_df['Cor1_transformed'] = data_df['Cor1'].fillna(0).astype(np.float32) / np.float32(6.0)
data_df.loc[data_df['Cor1_transformed'] > 1.0, ['Cor1_transformed']] = np.float32(1.0)
data_df['Cor2_transformed'] = data_df['Cor2'].fillna(0).astype(np.float32) / np.float32(6.0)
data_df.loc[data_df['Cor2_transformed'] > 1.0, ['Cor2_transformed']] = np.float32(1.0)

if drop_transformed:
    data_df = data_df.drop(['Cor1', 'Cor2'], axis = 1)

In [ ]:
data_df['P1_transformed'] = np.log(data_df['P1'], dtype = np.float32) / 2
data_df['P2_transformed'] = np.log(data_df['P2'], dtype = np.float32) / 2

if drop_transformed:
    data_df = data_df.drop(['P1', 'P2'], axis = 1)

In [ ]:
data_df.to_csv('data_train_transformed.csv.gz', index = False, compression={'method': 'gzip'})

In [ ]:
px.histogram(np.log(data_df['P2'].iloc[:1_000_000]) / 2)

In [ ]:
px.histogram(np.log(data_df['P1'].iloc[:1_000_000]))

In [ ]:
data_df = pd.read_csv('./data.csv', sep = ';',
                      #nrows = 10_000_000, 
                      usecols = ['Id', 'Minute', 'Score1', 'Score2', 
                                 'A1', 'A2', 'DA1', 'DA2', 'Pos1', 'Pos2', 'Off1', 
                                 'Off2', 'On1', 'On2', 'YC1', 'YC2', 'RC1', 'RC2', 
                                 'Sub1', 'Sub2', 'Pen1', 'Pen2', 'Cor1', 'Cor2'])

In [ ]:
data_df.head(100)

In [ ]:
!head /content/info.csv

Id;BeginTime;Home;Away;League;Country;Type;Result1;Result2;Periods;HomeId;AwayId;LeagueId;Season;Round
358904;01.01.2018 22:45:00;Bromley;Ebbsfleet United;England National League;gb;;4;2;3:1,4:2;;;;;
330509;01.01.2018 22:00:00;Hapoel Bnei Sakhnin;Maccabi Tel Aviv;Israel Premier League;il;;0;1;0:0,0:1;;;;;
575487;01.01.2018 20:30:00;Aston Villa;Bristol City;England Championship;gb;;5;0;2:0,5:0;;;;;
338563;01.01.2018 20:30:00;Everton;Man Utd;England Premier League;gb;;0;2;0:0,0:2;;;;;
340310;01.01.2018 20:15:00;Bangor City;Llandudno;Wales Premier League;gb;;0;1;0:0,0:1;;;;;
577973;01.01.2018 20:13:49;JKU FC;Taifa Jang’ombe;World Club Friendlies;;;0;0;0:0,0:0;;;;;
577112;01.01.2018 20:00:00;Hapoel Kfar Saba;Hapoel Ramat Gan;Israel Leumit Liga;il;;0;0;0:0,0:0;;;;;
571744;01.01.2018 20:00:00;Maccabi Achi Nazareth;Beitar Tel Aviv Bat Yam;Israel Leumit Liga;il;;0;0;0:0,0:0;;;;;
571518;01.01.2018 20:00:00;Ironi Ramat Hasharon;Hapoel Hadera;Israel Leumit Liga;il;;0;4;0:2,0:4;;;;;


In [ ]:
info_df = pd.read_csv('./info.csv', sep = ';', usecols = ['Id', 'Result1', 'Result2', 'Periods'])

In [ ]:
info_df[['Period1', 'Period2', 'Period3', 'Period4']] = info_df['Periods'].str.split(',', expand = True)

In [ ]:
miss_p1_list = list(info_df[info_df['Period1'] == '']['Id'])

In [ ]:
period1_result_dict = info_df.set_index('Id')['Period1'].to_dict()

In [ ]:
data_df.loc[:, 'periods1'] =  [period1_result_dict[id] if id in period1_result_dict else '' for id in data_df['Id'].values]


In [ ]:
(data_df['periods1'] == '').sum()

4247

In [ ]:
!head /content/prem.csv

Id;IsPrem;P1;PX;P2;PR;PRTime;IsKO;KO1;KOX;KO2;KOR;KOTime;KOminute;KOscore
358904;1;2.6;3.2;2.8;0.0543;01.01.2018 22:20:17;0;;;;;;;
330509;1;9.5;4.5;1.364;0.0606;01.01.2018 21:58:29;0;;;;;;;
575487;1;2.4;3.3;3.3;0.0227;01.01.2018 20:09:47;0;;;;;;;
338563;1;5.75;3.75;1.7;0.0288;01.01.2018 19:51:18;0;;;;;;;
340310;1;1.571;4;5.75;0.0605;01.01.2018 20:03:11;0;;;;;;;
577973;1;2;3.25;3.3;0.1107;31.12.2017 20:10:35;0;;;;;;;
577112;1;1.8;3.4;4.333;0.0805;01.01.2018 18:14:51;0;;;;;;;
571744;1;2.45;3.2;2.7;0.091;01.01.2018 12:36:33;0;;;;;;;
571518;1;1.909;3.3;3.75;0.0935;01.01.2018 19:37:02;0;;;;;;;


In [ ]:
prem_df = pd.read_csv('./prem.csv', sep = ';')

In [ ]:
prem_df

,Id,IsPrem,P1,PX,P2,PR,PRTime,IsKO,KO1,KOX,KO2,KOR,KOTime,KOminute,KOscore
0,358904,1,2.600,3.200,2.800,0.0543,01.01.2018 22:20:17,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,330509,1,9.500,4.500,1.364,0.0606,01.01.2018 21:58:29,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,575487,1,2.400,3.300,3.300,0.0227,01.01.2018 20:09:47,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,338563,1,5.750,3.750,1.700,0.0288,01.01.2018 19:51:18,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,340310,1,1.571,4.000,5.750,0.0605,01.01.2018 20:03:11,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749195,6058161,1,2.750,4.000,2.000,0.1136,26.12.2022 9:54:23,1.0,2.875,4.0,2.0,0.0978,26.12.2022 10:01:18,1.0,0.0
749196,5404664,1,2.800,3.600,2.450,0.0431,26.12.2022 9:59:53,1.0,2.750,3.6,2.5,0.0414,26.12.2022 10:01:30,1.0,0.0
749197,6063457,1,1.166,6.000,11.000,0.1152,26.12.2022 7:39:37,1.0,1.181,6.0,10.0,0.1134,26.12.2022 7:42:01,1.0,0.0
749198,6063456,1,1.083,8.500,23.000,0.0845,26.12.2022 0:36:08,1.0,1.090,8.5,21.0,0.0827,26.12.2022 5:01:48,1.0,0.0


In [ ]:
prem_df[prem_df['IsPrem'] != 1]